# Generators: x = yield 42 
## And other applications

### David Stuebe

<a href="https://www.swipely.com/" target="_blank">www.swipely.com</a>

<a href="mailto:davidstuebe@swipely.com">davidstuebe@swipely.com</a>

March 2, 2015

##What is a generator?
___

```
    A kind of function that can return an intermediate result ("the next
    value") to its caller, but maintaining the function's local state so
    that the function can be resumed again right where it left off.
```
<a href="https://www.python.org/dev/peps/pep-0255/" target="_blank">PEP 255</a> introduced the generator object and the *yield* statement in version 2.2 of Python.

In [435]:
def fib():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a+b

In [436]:
function_result = fib()
print function_result

<generator object fib at 0x112be1b40>


---
The result of calling the gererator function is a generator object.
The generator object can be used as an iterator.

The generator is an iterator
---

In [437]:
g.__iter__() is g


True

In [438]:
zip(xrange(10),function_result)

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 5),
 (5, 8),
 (6, 13),
 (7, 21),
 (8, 34),
 (9, 55)]

In [439]:
(10, function_result.next())

(10, 89)

## Inside a generator
---
Inside the generator we can see that execution is paused after the *yield* and state is maintained between calls to next


In [440]:
def noisy_generator():
    print '  Initializing'
    print '  first yield'
    yield "A"
    print '  generator running...'
    print '  second yield'
    yield "B"
    print '  generator running...'
    print '  now what?'

In [441]:
g = noisy_generator()        
print 'Result 1: %s' % g.next()
print 'Result 2: %s' % g.next()
print 'Result 3: %s' % g.next()

  Initializing
  first yield
Result 1: A
  generator running...
  second yield
Result 2: B
  generator running...
  now what?


StopIteration: 

## Generator interface
---

In [442]:
help(g)

Help on generator object:

noisy_generator = class generator(object)
 |  Methods defined here:
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __iter__(...)
 |      x.__iter__() <==> iter(x)
 |  
 |  __repr__(...)
 |      x.__repr__() <==> repr(x)
 |  
 |  close(...)
 |      close() -> raise GeneratorExit inside generator.
 |  
 |  next(...)
 |      x.next() -> the next value, or raise StopIteration
 |  
 |  send(...)
 |      send(arg) -> send 'arg' into generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  throw(...)
 |      throw(typ[,val[,tb]]) -> raise exception in generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  gi_code
 |  
 |  gi_frame
 |  
 |  gi_running



## Generator interface: throw
---

In [443]:
help(g.throw)

Help on built-in function throw:

throw(...)
    throw(typ[,val[,tb]]) -> raise exception in generator,
    return next yielded value or raise StopIteration.



Looks fun - lets try it...

In [444]:
def accepting_generator():
    print 'Initializing'
    try:
        yield 'Hello'
        print 'generator running...'
    except StandardError as e:
        print "Error Message: %s" % e
        yield 'World'

g = accepting_generator()
g.next()

Initializing


'Hello'

In [445]:
g.throw(StandardError, 'Foo bar baz')

Error Message: Foo bar baz


'World'

In [446]:
g.next()

StopIteration: 

## Generator interface: close
---

In [447]:
help(g.close)

Help on built-in function close:

close(...)
    close() -> raise GeneratorExit inside generator.



In [448]:
def closeable():
    try:
        yield 1
        yield 2
    except GeneratorExit:
        print 'closing'
g = closeable()
g.next()

1

In [449]:
g.close()

closing


In [450]:
g.next()

StopIteration: 

## Generator interface: send
---

In [451]:
help(g.send)

Help on built-in function send:

send(...)
    send(arg) -> send 'arg' into generator,
    return next yielded value or raise StopIteration.



In [452]:
g = fib()
g.send('foo')

TypeError: can't send non-None value to a just-started generator

so *next()* is really just *send(None)* 

## Lets try that again...

In [453]:
g = fib()
g.send(None)

1

In [454]:
g.send(None)

1

What is *send('foo')* ?

In [455]:
g.send('foo')

2

Where did it go?

## What is generator.send?
```
    Coroutines are a natural way of expressing many algorithms, such as
    simulations, games, asynchronous I/O, and other forms of event-
    driven programming or co-operative multitasking.  Python's generator
    functions are almost coroutines -- but not quite -- in that they
    allow pausing execution to produce a value, but do not provide for
    values or exceptions to be passed in when execution resumes.  They
    also do not allow execution to be paused within the "try" portion of
    try/finally blocks, and therefore make it difficult for an aborted
    coroutine to clean up after itself.
```
<a href="https://www.python.org/dev/peps/pep-0342/" target="_blank">PEP 342</a> added *close*, *send* and *throw* to the generator in version 2.5 of python and made *yield* an expresion rather than a statement.


In [456]:
def adder(val):
    x = 0
    while True:
        x = yield val+x
        
g = adder(5)
g.send(None)

5

In [457]:
g.send(2)

7

In [458]:
g.send(6)
        

11

Inside a generator part 2
---

In [459]:
def noisy_coroutine():
    print '  Initializing'
    print '  first yield'
    received = yield "A"
    print '  generator running after receiving: %s' % received
    print '  second yield'
    received = yield "B"
    print '  generator running after receiving: %s' % received
    print '  now what?'

In [460]:
g = noisy_coroutine()
g.send(None)

  Initializing
  first yield


'A'

In [461]:
g.send(1)


  generator running after receiving: 1
  second yield


'B'

In [462]:
g.send(2)

  generator running after receiving: 2
  now what?


StopIteration: 

In [463]:
def foo():
    x = yield
    y = yield
    z = yield
    yield x, y, z
    
g = foo()
g.next()
g.send(1)
g.send(2)
print g.send(3)

(1, 2, 3)


Calling next to start the generator is a pain
___
Lets fix that with a decorator...

In [464]:
def consumer(func):
    def wrapper(*args,**kw):
        gen = func(*args, **kw)
        gen.next()
        return gen
    wrapper.__name__ = func.__name__
    wrapper.__doc__  = func.__doc__
    return wrapper

In [465]:
g = consumer(adder)(4)
print g.send(11)
print g.send(2)

15
6


In [466]:
@consumer
def subtractor(val):
    '''A generator that subtracts numbers from a value'''
    x = 0
    while True:
        x = yield x - val

g = subtractor(8)
g.send(15)


7

In [467]:
help(subtractor)

Help on function subtractor in module __main__:

subtractor(*args, **kw)
    A generator that subtracts numbers from a value



## Lets try and do something interesting
___
So far we have looked at simple examples but we can use generators for:

* Iteration

* Data flow

* Concurrancy

* Reformulate control flow

## Iteration (a la <a href="http://www.dabeaz.com/coroutines/" target="_blank">David Beazley</a>)
___
An example of setting up a processing pipeline with generators

In [418]:
# From http://www.dabeaz.com/coroutines/pipeline.py
def grep(pattern,lines):
    for line in lines:
        if pattern in line:
             yield line

import time
def follow(thefile):
    thefile.seek(0,2)      # Go to the end of the file
    while True:
         line = thefile.readline()
         if not line:
             time.sleep(0.1)    # Sleep briefly
             continue
         yield line

# Set up a processing pipe : tail -f | grep python
with open("access-log") as logfile:
    loglines = follow(logfile)
    pylines  = grep("python",loglines)

    # Pull results out of the processing pipeline
    for line in pylines:
        print line,

217.237.150.206 - - [02/Apr/2015:00:47:42 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE060.HTM HTTP/1.0" 200 1686
129.106.32.126 - - [02/Apr/2015:00:47:43 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE006.HTM HTTP/1.0" 200 1254
66.91.239.214 - - [02/Apr/2015:00:47:44 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE014.HTM HTTP/1.0" 200 1232
217.219.18.80 - - [02/Apr/2015:00:47:46 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE090.HTM HTTP/1.0" 200 2001


KeyboardInterrupt: 

## Data Flow (a la <a href="http://www.dabeaz.com/coroutines/" target="_blank">David Beazley</a>)
___
An example of setting up a similar pipeline with coroutines

In [423]:
import time
def follow(thefile, target):
    thefile.seek(0,2)      # Go to the end of the file
    while True:
         line = thefile.readline()
         if not line:
             time.sleep(0.1)    # Sleep briefly
             continue
         target.send(line)

@consumer
def grep(pattern, target):
    print "Looking for %s" % pattern
    while True:
        line = (yield)
        if pattern in line:
            target.send(line),

@consumer
def printer():
    while True:
         line = (yield)
         print line,

with open("access-log") as logfile:
    follow(logfile, grep('python',printer() ) )

Looking for python
66.249.65.37 - - [02/Apr/2015:00:51:57 -0600] "GET /papers/python.html HTTP/1.1" 404 133
128.135.125.245 - - [02/Apr/2015:00:51:58 -0600] "GET /python/tutorial/beazley_intro_python/intropy.pdf HTTP/1.0" 304 -
194.105.57.11 - - [02/Apr/2015:00:52:01 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE002.HTM HTTP/1.0" 200 1352
189.141.19.88 - - [02/Apr/2015:00:52:03 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE096.HTM HTTP/1.0" 200 1671
123.190.193.8 - - [02/Apr/2015:00:52:03 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE059.HTM HTTP/1.0" 200 1694


KeyboardInterrupt: 

## Are generators fast for data flow?
___
Lets start with Beazely's <a href="http://www.dabeaz.com/coroutines/benchmark.py" target="_blank">Benchmark</a> example 

In [468]:
# An object
class GrepHandler(object):
    def __init__(self,pattern, target):
        self.pattern = pattern
        self.target = target
    def send(self,line):
        if self.pattern in line:
            self.target.send(line)

# A coroutine
@consumer
def grep(pattern,target):
    while True:
        line = (yield)
        if pattern in line:
            target.send(line)

# A null-sink to send data
@consumer
def null(): 
    while True: item = (yield)

# A benchmark
line = 'python is nice'
p1   = grep('python',null())          # Coroutine
p2   = GrepHandler('python',null())   # Object

from timeit import timeit

print "coroutine:", timeit("p1.send(line)", "from __main__ import line, p1")

print "object:", timeit("p2.send(line)", "from __main__ import line, p2")

coroutine: 0.320209026337
object: 0.388246059418


## Lets call send a few more times...
___
The idea for this benchmark comes from a stack overflow question <a href="http://stackoverflow.com/questions/15434420/a-faster-nested-tuple-to-list-and-back" target="_blank">a-faster-nested-tuple-to-list-and-back</a>

```
I'm trying to perform tuple to list and list to tuple conversions on nested sequences of unknown depth and shape. The calls are being made hundreds of thousands of times, which is why I'm trying to squeeze out as much speed as possible.
```

First, lets define a function to make some test data...


In [473]:
def make_test(m, n):
  return [[range(m), make_test(m, n-1)] for i in range(n)]
make_test(2,3)

[[[0, 1], [[[0, 1], [[[0, 1], []]]], [[0, 1], [[[0, 1], []]]]]],
 [[0, 1], [[[0, 1], [[[0, 1], []]]], [[0, 1], [[[0, 1], []]]]]],
 [[0, 1], [[[0, 1], [[[0, 1], []]]], [[0, 1], [[[0, 1], []]]]]]]

In [474]:
def list2tuple(a):
  return tuple((list2tuple(x) if isinstance(x, list) else x for x in a))

def tuple2list(a):
  return list((tuple2list(x) if isinstance(x, tuple) else x for x in a))

list2tuple(make_test(2,3))

(((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))),
 ((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))),
 ((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))))

<a href="http://stackoverflow.com/a/15434789/2136991" target="_blank">From HYRY's answer</a>

##Now lets try a solution using coroutines
___

In [485]:
@consumer
def cotuple2list():
  """converts tuples to lists"""
  result = None
  while True:
    (tup, co_pool) = (yield result)
    result = list(tup)
    for (i,x) in enumerate(result):
      if isinstance(x,tuple):
        result[i] = co_pool[0].send((x, co_pool[1:]))


@consumer
def colist2tuple():
  """Convertes lists to tuples"""
  result = None
  while True:
    (lst, co_pool) = (yield result)
    for (i,x) in enumerate(lst):
      if isinstance(x,list):
        lst[i] = co_pool[0].send((x, co_pool[1:]))
    result = tuple(lst)


<a href="http://stackoverflow.com/a/15435926/2136991" target="_blank">From my answer</a>

## Lets timeit!
___

In [487]:
t = make_test(breadth, depth)
%timeit colist2tuple_pool[0].send((t, colist2tuple_pool[1:]))

The slowest run took 55252.67 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.3 µs per loop


Okay, that didn't work...

In [488]:
import timeit
number = 10
repeat = 5
depth = 7
breadth = 25
print "list2tuple\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('list2tuple(t)', setup='from __main__ import list2tuple, make_test, depth, breadth; t = make_test(breadth, depth)', number=number, repeat=repeat)]

colist2tuple_pool = [colist2tuple() for i in xrange(breadth+1) ]
print "colist2tuple_pool\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('colist2tuple_pool[0].send((t, colist2tuple_pool[1:]))', setup='from __main__ import colist2tuple_pool, make_test, depth, breadth; t = make_test(breadth, depth)', number=number, repeat=repeat)]


list2tuple
time:  ['0.13708', '0.15126', '0.13760', '0.12958', '0.12790']
colist2tuple_pool
time:  ['0.01194', '0.01210', '0.01241', '0.01220', '0.01219']


Now that looks more interesting!

In [ ]:
## Lets try HYRY's Cython
___


In [575]:
import cython

list2tuple3, tuple2list3 = cython.inline(
    """
    @cython.profile (True)
    def list2tuple3(a):
        return tuple([list2tuple3(x) if type(x)==list else x for x in a])

    @cython.profile (True)
    def tuple2list3(a):
        return [tuple2list3(x) if type(x)==tuple else x for x in a]
    """
    ).values() # it returns a dict of named functions

Compiling /Users/davidstuebe/Library/Caches/cython/inline/_cython_inline_838386d3f99a608117807ccda8cfeebc.pyx because it changed.
Cythonizing /Users/davidstuebe/Library/Caches/cython/inline/_cython_inline_838386d3f99a608117807ccda8cfeebc.pyx


In [576]:
print "colist2tuple_pool\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('colist2tuple_pool[0].send((t, colist2tuple_pool[1:]))', setup='from __main__ import colist2tuple_pool, make_test, depth, breadth; t = make_test(breadth, depth)', number=number, repeat=repeat)]

print "list2tuple3\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('list2tuple3(t)', setup='from __main__ import list2tuple3, make_test, depth, breadth; t = make_test(breadth, depth)', number=number, repeat=repeat)]

colist2tuple_pool
time:  ['0.01400', '0.01310', '0.01315', '0.01294', '0.01290']
list2tuple3
time:  ['0.02258', '0.02360', '0.02331', '0.02264', '0.02310']


### What the heck - it is faster than cython!

## Lets try profiling?
___

In [584]:
import cProfile
t = make_test(10, depth)

In [585]:
cProfile.run('list2tuple(t);')

         438372 function calls (178098 primitive calls) in 0.220 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  41098/1    0.109    0.000    0.219    0.219 <ipython-input-474-19d1758e8e98>:1(list2tuple)
 219185/8    0.073    0.000    0.219    0.027 <ipython-input-474-19d1758e8e98>:2(<genexpr>)
        1    0.001    0.001    0.220    0.220 <string>:1(<module>)
   178087    0.036    0.000    0.036    0.000 {isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [586]:
cProfile.run('list2tuple3(t);')


         41100 function calls (3 primitive calls) in 0.040 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.040    0.040 <string>:1(<module>)
  41098/1    0.039    0.000    0.039    0.039 _cython_inline_838386d3f99a608117807ccda8cfeebc.pyx:8(list2tuple3)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [587]:
cProfile.run('colist2tuple_pool[0].send((t, colist2tuple_pool[1:]));')

         260285 function calls (178091 primitive calls) in 0.125 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  41098/1    0.087    0.000    0.125    0.125 <ipython-input-485-f23827ad9191>:13(colist2tuple)
        1    0.000    0.000    0.125    0.125 <string>:1(<module>)
   178087    0.032    0.000    0.032    0.000 {isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  41098/1    0.006    0.000    0.125    0.125 {method 'send' of 'generator' objects}




This doesn't help me much? Any thoughts?

## Lets try a dot product
___


In [608]:
import numpy
import itertools
import time
 
# make some data
n = 1000000
a = numpy.random.randn(n)
b = numpy.random.randn(n)
    

In [614]:
print "Numpy Dot Product\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('numpy.dot(a,b)', setup='from __main__ import numpy, a, b', number=number, repeat=repeat)]

Numpy Dot Product
time:  ['0.00117', '0.00098', '0.00081', '0.00082', '0.00084']


In [613]:
@consumer
def mult(target=None):
    if target is None:
        raise RuntimeError('Must specify targe coroutine!')
 
    result = numpy.float64(0.0)
    while True:
        (a, b) = (yield result)
        result = target.send(a*b)
        
@consumer
def add():
    result = numpy.float64(0.0)
    while True:
        m = (yield result)
        result += m
        
dot_product_process = mult(add())
 
def time_co_loop(it):
    for a_val,b_val in it: 
        dot_product = dot_product_process.send((a_val, b_val))

print "Coroutine Dot Product\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('time_co_loop(it)', setup='from __main__ import numpy, time_co_loop, a, b; it = numpy.nditer([a, b])', number=number, repeat=repeat)]



Coroutine Dot Product
time:  ['0.10584', '0.11044', '0.10758', '0.11602', '0.10354']


In [616]:
def time_loop(it):
    result = numpy.float64(0.0)
    for (a_val,b_val) in it:
        result += a*b
    return result

print "Naive Dot Product\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('time_loop(it)', setup='from __main__ import numpy, time_loop, a, b; it = numpy.nditer([a, b])', number=number, repeat=repeat)]

KeyboardInterrupt: 

In [615]:
# Coroutine processes without using return...
@consumer
def mult_noresult(target=None):
    if target is None:
        raise RuntimeError('Must specify targe coroutine!')
    while True:
        (a, b) = (yield)
        result = target.send(a*b)
                
@consumer
def add_noresult(result):
    while True:
        m = (yield)
        result[0] += m
 
 
result = numpy.zeros((1,))
dot_product_process_noresult = mult_noresult(add_noresult(result))
  
def time_co_result_loop(it):
    for a_val,b_val in it: 
        dot_product = dot_product_process_noresult.send((a_val, b_val))

print "Coroutine Dot Product\ntime:  %s" % ["%0.5f" % (v/number) for v in timeit.repeat('time_co_result_loop(it)', setup='from __main__ import numpy, time_co_result_loop, a, b; it = numpy.nditer([a, b])', number=number, repeat=repeat)]


Coroutine Dot Product
time:  ['0.12511', '0.12220', '0.12126', '0.12447', '0.12446']


### Based on David M. Beazley
### Generators: The Final Frontier
<a href="http://www.dabeaz.com/finalgenerator/" target="_blank">http://www.dabeaz.com/finalgenerator/</a>

### A Curious Course on Coroutines and Concurrency
<a href="http://www.dabeaz.com/coroutines/" target="_blank">http://www.dabeaz.com/coroutines/</a>
